## Project 3: Numerical Methods for Differential Equations

## Task 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import eigs
from mpl_toolkits.mplot3d import axes3d
from scipy import meshgrid

In [ ]:
def toeplitz_like(dim):
    return csr_matrix((diags([1, -2, 1], [-1, 0, 1], shape=(dim, dim))) / (dim + 1)**2)

In [87]:
def add(y_new):
    y_new = np.concatenate(([0], y_new, [0]))
    y_new = np.concatenate((y_new, [0]))
    return y_new

In [88]:
def euler_step(A: np.array, y_n: np.array, h: float):
    y_new = y_n + h * A @ y_n
    return add(y_new)

### Task 1.1

In [85]:
N = 999
M = 999
tend = 100
xx = np.linspace(0, 1, N+2)
tt = np.linspace(0, tend, M+1)
xv, yv = np.meshgrid(xx, tt)
alpha, beta = 0, 0

In [ ]:
def f(x):
    return x*(x-1)

In [86]:
print(len(xv[:,0]))

xv[:, 0] = f(xx)


1000


ValueError: could not broadcast input array from shape (1001,) into shape (1000,)

In [84]:
for i in range(M):
    xv[:, i + 1] = euler_step(toeplitz_like(N), xv[:, 1:-1], 1 / (N+1))

ValueError: dimension mismatch